In [1]:
!pip install peft datasets transformers numpy scikit-learn sentencepiece wandb

In [2]:
from peft import get_peft_model, LoraConfig
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import cohen_kappa_score
import os

os.environ["WANDB_LOG_MODEL"] = "end"

In [3]:
dataset = load_dataset("data", data_files={"train": "train.csv"})
dataset = dataset.rename_column('score', 'labels')
dataset = dataset.map(lambda x: {'labels': x['labels'] - 1})
dataset = dataset['train'].train_test_split(test_size=0.1)

In [4]:
MODEL_PATH = "microsoft/deberta-v3-large"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
tokenized_datasets = dataset.map(lambda x: tokenizer(x["full_text"], truncation=True, max_length=None), batched=True, remove_columns=['essay_id', 'full_text'])

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    config=AutoConfig.from_pretrained(MODEL_PATH, num_labels=len(set(tokenized_datasets['train']['labels']))),
)
model = get_peft_model(
    model,
    LoraConfig(
        task_type="SEQ_CLS", r=8, lora_alpha=16, lora_dropout=0.1
    )
)
# for param in model.deberta.parameters():
#     param.requires_grad = False

/Users/alistairkeiller/grading/env/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/15576 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1731 [00:00<?, ? examples/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="trainer",
    load_best_model_at_end=True,
    metric_for_best_model="eval_qwk",
    evaluation_strategy="steps",
    auto_find_batch_size=True,
    tf32=True,
    optim="adamw_torch_fused",
    report_to="wandb",
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"qwk": cohen_kappa_score(predictions, labels, weights='quadratic')}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Epoch,Training Loss,Validation Loss
